In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# prep
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import binarize, LabelEncoder, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

# Validation libraries
from sklearn import metrics
from sklearn.metrics import accuracy_score, mean_squared_error, precision_recall_curve
from sklearn.model_selection import cross_val_score

from aif360.sklearn.preprocessing import ReweighingMeta
from aif360.sklearn.inprocessing import AdversarialDebiasing
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, PostProcessingMeta
from aif360.sklearn.metrics import disparate_impact_ratio, average_odds_error, generalized_fpr
from aif360.sklearn.metrics import generalized_fnr, difference

In [60]:
treatment_categorized = pd.read_csv('treatment_categorized.csv',sep=';')
treatment_categorized.head()
treatment_categorized.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942 entries, 0 to 941
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Age                        942 non-null    object
 1   Country                    942 non-null    object
 2   self_employed              942 non-null    object
 3   family_history             942 non-null    object
 4   work_interfere             942 non-null    object
 5   no_employees               942 non-null    object
 6   remote_work                942 non-null    object
 7   tech_company               942 non-null    object
 8   benefits                   942 non-null    object
 9   care_options               942 non-null    object
 10  wellness_program           942 non-null    object
 11  seek_help                  942 non-null    object
 12  anonymity                  942 non-null    object
 13  leave                      942 non-null    object
 14  mental_hea

In [61]:
feature_cols = ['Age', 'label_value', 'Country', 'Inflation', 'Employment']
X = treatment_categorized[feature_cols]
y = treatment_categorized.sex
X.head()

,Age,label_value,Country,Inflation,Employment
0,forties,0,'United States',low,high
1,twenties,1,'United States',low,high
2,thirties,0,'United States',low,high
3,forties,1,'United States',low,high
4,forties,1,'United States',low,high


In [62]:
le = preprocessing.LabelEncoder()
le.fit(X[['Age']])
X['Age'] = le.transform(X[['Age']])
# le.fit(X[['sex']])
# X['sex'] = le.transform(X[['sex']])
le.fit(X[['Country']])
X['Country'] = le.transform(X[['Country']])
le.fit(X[['Inflation']])
X['Inflation'] = le.transform(X[['Inflation']])
le.fit(X[['Employment']])
X['Employment'] = le.transform(X[['Employment']])

/usr/local/anaconda3/envs/IREI/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/anaconda3/envs/IREI/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-62-581a635c6be5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'] = le.transform(X[['Age']])
/usr/local/anaconda3/envs/IREI/lib/python3.9/site-packages/sklearn/utils/validation.py:63

In [63]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, train_size=0.7, random_state=1234567)

In [64]:
y_pred = LogisticRegression(solver='lbfgs').fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, y_pred)

0.7314487632508834

In [65]:
disparate_impact_ratio(y_test, y_pred, prot_attr='sex')

ValueError: Some of the attributes provided are not present in the dataset. Expected a subset of:
[]
Got:
['sex']

In [58]:
average_odds_error(y_test, y_pred, prot_attr='sex')

ValueError: Some of the attributes provided are not present in the dataset. Expected a subset of:
[]
Got:
['sex']